In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-gpu")
    # Ne pas activer le mode headless pour que tout se charge correctement
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(616, 970)  # Taille trouvée via ton enregistrement
    driver.set_page_load_timeout(60)
    return driver

def click_voir_plus_until_end(driver, wait_time=2, max_clicks=100):
    wait = WebDriverWait(driver, 10)
    for i in range(max_clicks):
        try:
            btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[title='Voir plus']")))
            print(f"🟢 Clic {i+1} sur 'Voir plus'")
            driver.execute_script("arguments[0].scrollIntoView(true);", btn)
            time.sleep(0.5)
            btn.click()
            time.sleep(wait_time)
        except:
            print("✅ Plus de bouton 'Voir plus'")
            break

def extract_agences_from_html(html):
    soup = BeautifulSoup(html, "html.parser")
    agences = []

    for article in soup.select("article.b-result"):
        lat = article.get("data-lat")
        lng = article.get("data-lng")
        nom_tag = article.select_one("h2.b-result__name")
        adresse_tags = article.select("p.b-result__address")

        if nom_tag and len(adresse_tags) >= 2:
            nom = nom_tag.text.strip()
            adresse_ligne1 = adresse_tags[0].text.strip()
            adresse_ligne2 = adresse_tags[1].text.strip()
            adresse = f"{adresse_ligne1}, {adresse_ligne2}"
            code_postal = adresse_ligne2.split()[0]

            agences.append({
                "nom": nom,
                "adresse": adresse,
                "code_postal": code_postal,
                "latitude": lat,
                "longitude": lng
            })

    return agences

def scrape_region(region_slug, output_dir="agences_bnp"):
    url = f"https://nos-agences.mabanque.bnpparibas/{region_slug}"
    driver = setup_driver()

    print(f"\n🌍 Région : {region_slug}")
    driver.get(url)
    
    
    try:
        wait = WebDriverWait(driver, 5)
        accept_btn = wait.until(EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button")))
        accept_btn.click()
        print("🍪 Cookies acceptés")
        time.sleep(1)
    except:
        print("ℹ️ Pas de bannière de cookies détectée (ou déjà acceptée)")


    click_voir_plus_until_end(driver)
    html = driver.page_source
    driver.quit()

    agences = extract_agences_from_html(html)
    df = pd.DataFrame(agences)
    df.drop_duplicates(subset=["nom", "adresse"], inplace=True)

    os.makedirs(output_dir, exist_ok=True)
    path = os.path.join(output_dir, f"{region_slug}.csv")
    df.to_csv(path, index=False)
    print(f"✅ {len(df)} agences sauvegardées dans {path}")

def scrape_all_regions():
    regions = [
        "bourgogne-franche-comte",
        "ile-de-france",
        "hauts-de-france",
        "occitanie",
        "bretagne",
        "auvergne-rhone-alpes",
        "nouvelle-aquitaine",
        "pays-de-la-loire",
        "provence-alpes-cote-d-azur",
        "grand-est",
        "normandie",
        "centre-val-de-loire"
    ]
    for region in regions:
        scrape_region(region)

if __name__ == "__main__":
    scrape_all_regions()



🌍 Région : bretagne
ℹ️ Pas de bannière de cookies détectée (ou déjà acceptée)
🟢 Clic 1 sur 'Voir plus'
🟢 Clic 2 sur 'Voir plus'
🟢 Clic 3 sur 'Voir plus'
🟢 Clic 4 sur 'Voir plus'
🟢 Clic 5 sur 'Voir plus'
🟢 Clic 6 sur 'Voir plus'
🟢 Clic 7 sur 'Voir plus'
🟢 Clic 8 sur 'Voir plus'
🟢 Clic 9 sur 'Voir plus'
✅ Plus de bouton 'Voir plus'
✅ 92 agences sauvegardées dans agences_bnp\bretagne.csv

🌍 Région : auvergne-rhone-alpes
ℹ️ Pas de bannière de cookies détectée (ou déjà acceptée)
🟢 Clic 1 sur 'Voir plus'
🟢 Clic 2 sur 'Voir plus'
🟢 Clic 3 sur 'Voir plus'
🟢 Clic 4 sur 'Voir plus'
🟢 Clic 5 sur 'Voir plus'
🟢 Clic 6 sur 'Voir plus'
🟢 Clic 7 sur 'Voir plus'
🟢 Clic 8 sur 'Voir plus'
🟢 Clic 9 sur 'Voir plus'
🟢 Clic 10 sur 'Voir plus'
🟢 Clic 11 sur 'Voir plus'
🟢 Clic 12 sur 'Voir plus'
🟢 Clic 13 sur 'Voir plus'
🟢 Clic 14 sur 'Voir plus'
🟢 Clic 15 sur 'Voir plus'
🟢 Clic 16 sur 'Voir plus'
🟢 Clic 17 sur 'Voir plus'
🟢 Clic 18 sur 'Voir plus'
🟢 Clic 19 sur 'Voir plus'
🟢 Clic 20 sur 'Voir plus'
🟢 Clic 21